In [1]:
%load_ext autoreload

import threading
import time
import numpy as np

**First Demonstrate the Race Condition**

In [2]:
x = 0

def increment():
    global x
    x += 1
 
def thread1_task():
    for _ in range(100000):
        increment()
 
def thread2_task():
    for _ in range(100000):
        increment()
 
def main_task():
    global x
    x = 0
   
    t1 = threading.Thread(target=thread1_task)
    t2 = threading.Thread(target=thread2_task)
 
    t1.start()
    t2.start()
 
    t1.join()
    t2.join()

for i in range(10):
    main_task()
    print("Iteration {0}: x = {1}".format(i,x))

Iteration 0: x = 200000
Iteration 1: x = 200000
Iteration 2: x = 184488
Iteration 3: x = 200000
Iteration 4: x = 131293
Iteration 5: x = 200000
Iteration 6: x = 200000
Iteration 7: x = 200000
Iteration 8: x = 160228
Iteration 9: x = 163083


**Implement the 1st "Solution"**

In [3]:
class Solution_1:

    def __init__(self):
        self.turn = 0

    def lock(self, thread_ID):
        while self.turn != thread_ID: pass  # spin

    def unlock(self, thread_ID):
        if thread_ID == 1: self.turn = 0    
        else: self.turn = 1

**Use Solution 1 with Race Condition**

In [4]:
x = 0
lock = Solution_1()

def increment():
    global x
    x += 1
 
def thread1_task(lock, my_num):
   
    for _ in range(1000):
        lock.lock(my_num)       # lock before critical section
        increment()
        lock.unlock(my_num)     # unlock after critical section
 
def thread2_task(lock, my_num):
   
    for _ in range(1000):
        lock.lock(my_num)       # lock before critical section
        increment()
        lock.unlock(my_num)     # unlock after critical section
 
def main_task():
    global x
    global lock
 
    x = 0
   
    t1 = threading.Thread(target=thread1_task, args=(lock, 0, ))
    t2 = threading.Thread(target=thread2_task, args=(lock, 1, ))
 
    t1.start()
    t2.start()
 
    t1.join()
    t2.join()

**Test the First "Solution"**

In [8]:
for i in range(10):
    main_task()
    print("Iteration {0}: x = {1}".format(i,x))

Iteration 0: x = 2000
Iteration 1: x = 2000
Iteration 2: x = 2000
Iteration 3: x = 2000
Iteration 4: x = 2000
Iteration 5: x = 2000
Iteration 6: x = 2000
Iteration 7: x = 2000
Iteration 8: x = 2000
Iteration 9: x = 2000


Mutual exclusion acheived! with only *minor* performance consequences. I had to reduce iterations to 1000 (2 orders of magnitude) to get it to run in reasonable time. 

**Implement the Second "Solution"**

In [2]:
class Solution_2:

    def __init__(self):
        self.flag = [False, False]

    def lock(self, thread_ID):
        
        if thread_ID == 0:
            self.flag[0] = True
            while self.flag[1]: pass
        else:
            self.flag[1] = True
            while self.flag[0]: pass

    def unlock(self, thread_ID):

        if thread_ID == 0:
            self.flag[0] = False
        else:
            self.flag[1] = False

**Use Second "Solution" with Race Condition**

In [3]:
x = 0
lock = Solution_2()

def increment():
    global x
    x += 1
 
def thread1_task(lock, my_num):
   
    for _ in range(10000):
        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def thread2_task(lock, my_num):
   
    for _ in range(10000):
        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def main_task():
    global x
    global lock
 
    x = 0
   
    t1 = threading.Thread(target=thread1_task, args=(lock, 0, ))
    t2 = threading.Thread(target=thread2_task, args=(lock, 1, ))
 
    t1.start()
    t2.start()
 
    t1.join()
    t2.join()

In [4]:
for i in range(10):
    main_task()
    print("Iteration {0}: x = {1}".format(i,x))

Iteration 0: x = 20000


**Use Peterson's solution**

In [10]:
from peterson import Peterson

lock = Peterson()

x=0

def increment():
    global x
    x += 1
 
def thread1_task(lock, my_num):
   
    for _ in range(10000):

        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def thread2_task(lock, my_num):
   
    for _ in range(10000):

        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def main_task():
    global x
    global lock
 
    x = 0
   
    t1 = threading.Thread(target=thread1_task, args=(lock, 0, ))
    t2 = threading.Thread(target=thread2_task, args=(lock, 1, ))
 
    t1.start()
    t2.start()
 
    t1.join()
    t2.join()

In [11]:
for i in range(10):
    main_task()
    print("Iteration {0}: x = {1}".format(i,x))

Iteration 0: x = 20000
Iteration 1: x = 20000
Iteration 2: x = 20000
Iteration 3: x = 20000
Iteration 4: x = 20000
Iteration 5: x = 20000
Iteration 6: x = 20000
Iteration 7: x = 20000
Iteration 8: x = 20000
Iteration 9: x = 20000


**Use Bakery Algorithm**

In [16]:
from bakery import Bakery

x=0
lock = Bakery(2)

def increment():
    global x
    x += 1
 
def thread1_task(lock, my_num):
   
    for _ in range(10000):

        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def thread2_task(lock, my_num):
   
    for _ in range(10000):

        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def main_task():
    global x
    global lock
 
    x = 0
   
    t1 = threading.Thread(target=thread1_task, args=(lock, 0, ))
    t2 = threading.Thread(target=thread2_task, args=(lock, 1, ))
 
    t1.start()
    t2.start()
 
    t1.join()
    t2.join()

In [17]:
for i in range(10):
    main_task()
    print("Iteration {0}: x = {1}".format(i,x))

Iteration 0: x = 20000
Iteration 1: x = 20000
Iteration 2: x = 20000


KeyboardInterrupt: 

**Demonstrate Bakery Algorithm with N threads**

In [2]:
from bakery import Bakery

x=0
lock = Bakery(3)

def increment():
    global x
    x += 1
 
def thread1_task(lock, my_num):
   
    for _ in range(1000):

        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def thread2_task(lock, my_num):
   
    for _ in range(1000):

        lock.lock(my_num)
        increment()
        lock.unlock(my_num)

def thread3_task(lock, my_num):
   
    for _ in range(1000):

        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def main_task():
    global x
    global lock
 
    x = 0
   
    t1 = threading.Thread(target=thread1_task, args=(lock, 0, ))
    t2 = threading.Thread(target=thread2_task, args=(lock, 1, ))
    t3 = threading.Thread(target=thread3_task, args=(lock, 2, ))
 
    t1.start()
    t2.start()
    t3.start()
 
    t1.join()
    t2.join()
    t3.join()

In [3]:
for i in range(10):
    main_task()
    print("Iteration {0}: x = {1}".format(i,x))

Iteration 0: x = 3000
Iteration 1: x = 3000
Iteration 2: x = 3000
Iteration 3: x = 3000
Iteration 4: x = 3000
Iteration 5: x = 3000
Iteration 6: x = 3000
Iteration 7: x = 3000
Iteration 8: x = 3000
Iteration 9: x = 3000
